In [1]:
%reload_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

In [2]:
from IMLearn.utils import split_train_test
from IMLearn.learners.regressors import LinearRegression

from typing import NoReturn, Optional
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [ ]:
np.random.seed(0)
df = pd.read_csv("../datasets/house_prices.csv")

y = df['price']
X = df.drop('price', axis=1)
train_X, train_y, test_X, test_y = split_train_test(X, y, 0.75)

id - <b>not important for model</b>
date - <b>drop</b>
bedrooms - yes
bathrooms - yes
sqft_living - yes
sqft_lot - yes
floors - yes
waterfront - yes
view - yes
condition - yes
grade - 1 to 13 overall grade <b> maybe break down into other features</b>
sqft_above - yes
sqft_basement - yes
yr_built - yes
yr_renovated - yes
zipcode - <b> break into hot one encoding, maybe get rid of </b>
lat - <b>check!</b>
long - <b>check!</b>
sqft_living15 - neighbors avg sqft
sqft_lot15 - neighbors avg lot sqft

In [ ]:
X = train_X
y = train_y
train = X
train['price'] = y
train = train.dropna()
# train = train.drop('date', axis=1)
train = train[train['id'] != 0.0]
train = train[train['sqft_lot15'] > 0]
train = train[train['price'] > 0]

Adding one-hot encoding to zipcodes

In [ ]:
one_hot_zipcode_df = pd.get_dummies(train['zipcode'], prefix='zip')
train = train.join(one_hot_zipcode_df)
train = train.drop('zipcode', axis=1)

In [ ]:
train[train.duplicated(subset=['id'], keep=False)]

In [ ]:
train.describe(include='all')

In [ ]:
(train.isna().sum() / len(train)) * 100

In [ ]:
train[train.duplicated(subset=['id', 'date'], keep=False)]

In [ ]:
train.dtypes

### feature engineering

In [ ]:
# TODO number of bathrooms per bedrooms

In [ ]:
# age of the house
X['date'] = pd.to_datetime(X['date'], format='%Y%m%dT000000', errors='coerce')
X['house_age_yr'] = X['date'].dt.year - X['yr_built']

In [ ]:
X['sqft_ratio'] = X['sqft_living'] / X['sqft_lot']
X

In [ ]:
fig = px.scatter(x=X['grade'], y=X['price'])
fig.show()

In [ ]:
# a very cool graphic representation:
test = X[X['id'] != 0.0]
test = test[test['price'] > 0]
fig = px.scatter(test, x='long', y='lat', color='price', size='price')
fig.show()

lat: 47.620422
long: -122.349358

In [ ]:
def dist_from_center(lat, long):
    ref_lat, ref_long = 47.612619936344856, -122.20516535787827  # Bellevue Downtown Park
    R = 6371  # radius of the Earth in km
    phi1 = np.radians(lat)
    phi2 = np.radians(ref_lat)
    delta_phi = np.radians(ref_lat - lat)
    delta_lambda = np.radians(ref_long - long)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

In [ ]:
test['dist_from_center'] = test.apply(lambda row: dist_from_center(row['lat'], row['long']), axis=1)
px.scatter(test, x='dist_from_center', y='price').show()

In [ ]:
test['bed_bath_ratio'] = test['bathrooms'] / test['bedrooms']
px.scatter(test, x='bed_bath_ratio', y='price')

# Beggining

In [131]:
def dist_from_reference(lat, long):
    ref_lat, ref_long = 47.612619936344856, -122.20516535787827  # Bellevue Downtown Park
    R = 6371  # radius of the Earth in km
    phi1 = np.radians(lat)
    phi2 = np.radians(ref_lat)
    delta_phi = np.radians(ref_lat - lat)
    delta_lambda = np.radians(ref_long - long)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

In [190]:
np.random.seed(0)
df = pd.read_csv("../datasets/house_prices.csv")

# Question 1 - split data into train and test sets
y = df.loc[:, 'price']
X = df.drop('price', axis=1)
train_X, train_y, test_X, test_y = split_train_test(X, y, 0.75)

X, y = train_X, train_y

In [191]:

# Question 2 - Preprocessing of housing prices dataset
X.loc[:, 'price'] = y.loc[:]
# X = X[X['sqft_lot15'] > 0]
X.loc[:] = X.loc[X['sqft_lot15'] > 0]
X.dropna(inplace=True)
X.drop_duplicates(subset=['id'], inplace=True)

In [192]:

# FEATURE ENGINEERING:
# one-hot encoding for zipcodes
one_hot_zipcode_df = pd.get_dummies(X['zipcode'], prefix='zip')
X = X.join(one_hot_zipcode_df)
# age of the house
X.loc[:, 'date'] = pd.to_datetime(X['date'], format='%Y%m%dT000000', errors='coerce')
X.loc[:, 'house_age_yr'] = X['date'].dt.year - X['yr_built']
# square footage ratio
X['sqft_ratio'] = X['sqft_living'] / X['sqft_lot']
# distance from city center
X.loc[:, 'dist_from_center'] = X.apply(lambda row: dist_from_reference(row['lat'], row['long']), axis=1)
# bathroom to bedroom ratio
X.loc[:, 'bath_bed_ratio'] = X['bathrooms'] / X['bedrooms']
X.loc[:, 'bath_bed_ratio'] = X.loc[:, 'bath_bed_ratio'].replace([np.inf, -np.inf], np.nan)
X['bath_bed_ratio'].fillna(0, inplace=True)
# was house renovated
X['is_renovated'] = X['yr_renovated'].apply(lambda row: 1 if (row != 0.0) else 0)
# last column drops
X.drop(['date', 'id', 'lat', 'long', 'zipcode', 'yr_renovated'], axis=1, inplace=True)

# X = X.replace([np.inf, -np.inf], np.nan).dropna()

y = X.loc[:, 'price']
X.drop('price', axis=1, inplace=True)

In [193]:
pearson = X.loc[:, ~(X.columns.str.contains('^zip_', case=False) | X.columns.str.contains('^is_renovated', case=False))]

In [95]:
pearson

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,sqft_living15,sqft_lot15,house_age_yr,sqft_ratio,dist_from_center,bed_bath_ratio
4808,3.0,2.50,2540.0,6093.0,2.0,0.0,0.0,3.0,9.0,2540.0,0.0,1999.0,2540.0,5924.0,15.0,0.416872,10.441816,0.833333
19509,3.0,2.00,1820.0,6324.0,1.0,0.0,0.0,5.0,7.0,910.0,910.0,1945.0,1850.0,6440.0,70.0,0.287793,9.144469,0.666667
4309,2.0,1.75,2330.0,7280.0,1.0,0.0,0.0,3.0,7.0,1450.0,880.0,1982.0,1830.0,12178.0,33.0,0.320055,21.262605,0.875000
2604,3.0,2.00,1600.0,16530.0,1.0,0.0,0.0,5.0,7.0,1600.0,0.0,1967.0,1850.0,41006.0,47.0,0.096794,14.584472,0.666667
1783,3.0,1.75,1530.0,8028.0,1.0,0.0,0.0,3.0,7.0,1200.0,330.0,1967.0,1530.0,8028.0,48.0,0.190583,21.663563,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9200,3.0,1.00,1520.0,7800.0,1.0,0.0,0.0,4.0,7.0,1120.0,400.0,1967.0,1740.0,8000.0,47.0,0.194872,26.812588,0.333333
10550,4.0,2.25,3290.0,12329.0,1.5,0.0,0.0,4.0,10.0,3290.0,0.0,1968.0,3170.0,12329.0,46.0,0.266851,5.478008,0.562500
4568,4.0,1.00,1310.0,6000.0,1.5,0.0,0.0,3.0,7.0,1310.0,0.0,1940.0,1630.0,6000.0,74.0,0.218333,7.537632,0.250000
16767,3.0,2.00,2060.0,4040.0,1.0,0.0,0.0,4.0,8.0,1120.0,940.0,1947.0,1500.0,4000.0,67.0,0.509901,13.296517,0.666667


for i, feature in enumerate(pearson, start=1):
    # cov = np.cov(X[feature], y) / (np.std(X[feature]) * np.std(y))
    fig = px.scatter(pd.DataFrame({'x': pearson[feature], 'y': y}), x='x', y='y')
    fig.write_html(str(i) + ".html")

In [96]:
def _config_plot(fig, feature, cov):
    fig.update_traces(
        marker=dict(
            size=5,
            symbol="diamond",
            color="#0033cc",
            opacity=0.7,
            line=dict(
                color='white',
                width=0.2
            )
        )
    )
    fig.update_layout(
        title={
            "text": f"Pearson correlation between {feature} and Response. <br>Pearson Correlation: {np.round(cov, 3)}",
            "x": 0.5, "y": 0.95},
        template="simple_white",
        font_color="black",
        title_font_family="Helvetica",
        title_font_color="black",
    )

In [178]:
for i, feature in enumerate(pearson):
    print(X[feature].dtype)
    p_corr = np.cov(X[feature], y)[0][1] / (np.std(X[feature]) * np.std(y))
    fig = px.scatter(pd.DataFrame({'x': pearson[feature], 'y': y}), x='x', y='y',
                     labels={'x': f'{feature} value', 'y': 'Response values'}, trendline='ols',
                     trendline_color_override='black')
    _config_plot(fig, feature, p_corr)
    # fig.write_html()

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


pearson.hist(bins='auto', figsize=(20, 20))